<a href="https://colab.research.google.com/github/tiagopessoalima/ED2/blob/main/Semana_14_(ED2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **B-Trees e Heaps: Estruturas para Armazenamento e Prioridade**

## **Introdução às B-Trees e Heaps**

As B-Trees e Heaps são estruturas de dados especializadas, projetadas para atender a necessidades específicas em sistemas computacionais. Enquanto as B-Trees são amplamente utilizadas em sistemas que requerem acesso eficiente a grandes volumes de dados armazenados em disco, como bancos de dados e sistemas de arquivos, os Heaps são ideais para gerenciar prioridades, sendo fundamentais em algoritmos como filas de prioridade e o Heap Sort.

## **O que são B-Trees?**

Uma B-Tree de ordem $(m)$ é uma árvore balanceada projetada para minimizar acessos a disco, permitindo que cada nó contenha múltiplas chaves (até $(m-1)$) e múltiplos filhos (até $(m)$). Suas principais propriedades são:

- Cada nó, exceto a raiz, contém pelo menos $(\lceil m/2 \rceil - 1)$ chaves.
- Todos os nós folhas estão no mesmo nível, garantindo balanceamento.
- As chaves em cada nó são ordenadas, e os filhos representam intervalos de valores.
- Operações como busca, inserção e remoção têm complexidade $O(\log_m n)$, onde $(n)$ é o número total de chaves.

Exemplo de Aplicação: Em bancos de dados como MySQL (com índices B+-Tree), as B-Trees permitem buscas rápidas em tabelas com milhões de registros, pois cada nó pode armazenar centenas de chaves, reduzindo o número de acessos ao disco.

Exemplo Visual (B-Tree de ordem 4):

```
       [20 | 40]
      /    |    \
[5 | 10] [25 | 30] [50 | 60]
```

## **O que são Heaps?**

Um Heap é uma árvore binária quase completa que satisfaz a propriedade de heap: em um max-heap, o valor de cada nó é maior ou igual ao de seus filhos; em um min-heap, é menor ou igual. Heaps são implementados eficientemente em arrays, com as seguintes características:

- Acessar o elemento de maior/menor prioridade (raiz) é ($O(1)$).
- Inserção e remoção têm complexidade ($O(\log n)$), onde ($n$) é o número de elementos.
- Usado em filas de prioridade, escalonamento de processos e no algoritmo Heap Sort.

Exemplo de Aplicação: Em sistemas operacionais, filas de prioridade baseadas em min-heaps gerenciam tarefas com base em sua urgência. O Heap Sort utiliza max-heaps para ordenar dados com complexidade ($O(n \log n)$).

Exemplo Visual (Max-Heap):

```
       90
      /  \
    70    80
   /  \   /
  50  60 20
```